# mopac2orca

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles_to_xyz(smiles):

    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.MMFFOptimizeMolecule(mol)

    xyz = ''
    conf = mol.GetConformer()
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        xyz += f"{atom.GetSymbol()} {pos.x} {pos.y} {pos.z}\n"

    return xyz

In [2]:
xyz_dict = {}

smiles_list = ["CCO", "CCCO"]

for smiles in smiles_list:
    xyz = smiles_to_xyz(smiles)
    xyz_dict[smiles] = xyz

for smiles, xyz in xyz_dict.items():
    print(f"SMILES: {smiles}\nXYZ:\n{xyz}\n")

SMILES: CCO
XYZ:
C -0.8870892544785829 0.17506438130212912 -0.012534849920828095
C 0.46048928281754536 -0.5155159315952142 -0.04653519550667649
O 1.4429650547754118 0.30726686900929995 0.5655716319307513
H -0.8474775449695346 1.127767875801999 -0.5508174316609016
H -1.6587817613201192 -0.45332672501352855 -0.4658420495184516
H -1.1769443399382888 0.40367644965162297 1.0183056465117932
H 0.7687124233445327 -0.7243278039255768 -1.0754595642185318
H 0.4194863470769616 -1.462072773783187 0.5001769007484058
H 1.478639792692034 1.141467658552465 0.06713491163443283


SMILES: CCCO
XYZ:
C 1.2614035820317906 0.10449317497169158 0.5970200582772502
C 0.2843156958624234 0.32748024441409695 -0.5482825344094758
C -0.9567676659833205 -0.5481774711755605 -0.42917047994944435
O -1.6855576530434582 -0.23661793639535864 0.7498980751430725
H 2.149711316920237 0.7297112669888961 0.46214988969150506
H 1.584961687759981 -0.9404786041657214 0.6375511606764974
H 0.8111350484598283 0.3630745936879234 1.56051660

In [3]:
import os

def extract_xyz_from_arc(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    start_index = None
    for index, line in enumerate(lines):
        if "FINAL GEOMETRY OBTAINED" in line:
            start_index = index + 4
            break
    if start_index is not None:
        xyz_data = ""
        for line in lines[start_index:]:
            if len(line.strip()) == 0:
                break
            cleaned_line = line.replace(" +1", "")
            parts = cleaned_line.split()
            xyz_data += f"{parts[0]}    {parts[1]}    {parts[2]}    {parts[3]}\n"
        return xyz_data
    return None

calc_1_dir = './calc_1'
xyz_dict = {}

for file_name in os.listdir(calc_1_dir):
    if file_name.endswith('.arc'):
        file_path = os.path.join(calc_1_dir, file_name)
        xyz_data = extract_xyz_from_arc(file_path)
        if xyz_data:
            key = file_name.replace('.arc', '')
            xyz_dict[key] = xyz_data



In [4]:
for key, xyz_data in xyz_dict.items():
    print(f"Key: {key}\nFirst lines of XYZ data:\n")
    lines = xyz_data.strip().split('\n')  
    for line in lines[:20]:  
        print(line)

Key: 63_conformation_optimized_0
First lines of XYZ data:

C    0.20811350    -2.02677727    1.46929517
C    1.29514894    -1.15751939    0.94678127
C    2.62349959    -1.58770938    0.98704955
C    3.64468799    -0.77006121    0.50907733
C    3.35824863    0.49027095    -0.01051158
C    2.04031454    0.94704468    -0.06288670
C    1.72575407    2.29532670    -0.60958769
C    1.02610728    0.10847050    0.41372880
O    -0.25731372    0.60305665    0.45038553
C    -1.05736583    0.16683416    -0.65519716
C    -2.50246797    0.23796711    -0.11167453
C    -3.34261814    1.22040985    -0.94256121
N    -3.06560452    -1.12828878    -0.09511963
H    0.33256548    -2.22103919    2.54387354
H    0.17896440    -2.99743746    0.95604937
H    -0.79403266    -1.57525527    1.34763297
H    2.85906709    -2.56729281    1.39650389
H    4.67547148    -1.11705743    0.54477713
H    4.16330132    1.12254077    -0.37665532
H    2.43110420    3.05632817    -0.24690473
Key: 63_conformation_optimized_1
Fir

# XYZ в ORCA

In [5]:
import os


atomic_electrons = {'H': 1, 'C': 6, 'N': 7, 'O': 8}


def count_electrons(xyz_data, charge):
    electron_count = 0
    for atom in xyz_data.split('\n'):
        parts = atom.split()
        if len(parts) >= 4 and parts[0] in atomic_electrons: 
            electron_count += atomic_electrons[parts[0]]
    electron_count -= int(charge)
    return electron_count


def get_multiplicity(electron_count):
    return 1 if electron_count % 2 == 0 else 2


orca_files_dir = './orca_files'
os.makedirs(orca_files_dir, exist_ok=True)

for file in os.listdir(orca_files_dir):
    file_path = os.path.join(orca_files_dir, file)
    if os.path.isfile(file_path):
        os.remove(file_path)

for key, xyz_data in xyz_dict.items():
    base_path = os.path.join(orca_files_dir, f"{key}")

    for charge in ["0", "1", "-1"]:
        electron_count = count_electrons(xyz_data, charge)
        mult = get_multiplicity(electron_count)
        file_path = f"{base_path}_{charge}.inp"
        with open(file_path, 'w') as fw:
            fw.write('! wB97x-d3 def2-TZVPD def2/J RIJCOSX SlowConv\n')
            fw.write('%pal\n')
            fw.write('nprocs 12\n')
            fw.write('end\n')
            fw.write('%scf\n')
            fw.write('maxiter 1500\n')
            fw.write('end\n')
            fw.write(f'* xyz {charge} {mult}\n')
            fw.write(xyz_data)
            fw.write('*\n')

